In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext magic_duckdb



In [ ]:
# Optional: uses keys.ini to load openai_key. You can also just set it directly.

"""Optional: Use a keys.ini file"""
import configparser
import os
import magic_duckdb
from magic_duckdb.extras import sql_ai

keyfile = f"{os.path.dirname(magic_duckdb.__file__)}/keys.ini"

keys = configparser.ConfigParser()
keys.read(keyfile)

openai_key = keys.get("keys", "OPENAI_KEY", fallback=None) # type: ignore

if openai_key is None:
    print(f"No OPENAI_KEY in {keyfile}")
sql_ai.OPENAI_KEY = openai_key
sql_ai.print_prompts = True

In [ ]:
%%dql
CREATE OR REPLACE TABLE xyz as SELECT concat('', x) from range(100) t(x);
CREATE OR REPLACE TABLE abc as SELECT * from range(50) y(a);

In [ ]:
%%dql -aichat write a duckdb query

 windowed cumulative 
sum of xyz where value is in abc

In [ ]:
%%dql -aichat why is this query so slow? 

select * from range(1000000000000) full outer join select * from range(100000000)

In [ ]:
%%dql -aichat reformat this query using sql formatting standards

select * from range(1000000000000) full outer join select * from range(100000000)

In [ ]:
# Testing outside the magics
import duckdb
from magic_duckdb.extras import sql_ai

con = duckdb.connect()

sql_ai.print_prompts=True
con.execute("create table xyz as select * from range(100) t(x)")
con.execute("create table abc as select * from range(100) y(a)")

r = sql_ai.ai_statement(connection=con, prompt="suggest", statement="a query that combines results from all tables")
print(r)

r = sql_ai.ai_statement(connection=con, prompt="fix using existing tables and reformat according to SQL Formatting Standard with capital letters and newlines but no trailing slashes", statement="selct cont(*) frm x join a ")
print(r)

In [ ]:
%%dql -aichat

create a table containing firstnames and birthdays for children in highschool and compute the average age